In [1]:
import pandas as pd
from credentials import pid_202_token
from credentials import redcap_api_url
import redcap


In [2]:
df = pd.read_excel("sample_data.xlsx")

In [3]:
def transform_refugee_data():
    """
    three transformations to refugee health data:
    
    1) columns changed to lower case
    2) create index titled record_id which is a copy of "aliennumber" column
    3) tbclass column is transformed via a dictionary match
    """

    df.columns = df.columns.str.lower()

    df["record_id"] = df["aliennumber"]
    df = df.set_index(keys = df["record_id"])


    tbclass = {
        "A":"A",
        "B1 (TI 07)":"B1",
        "B2 (TI 07)":"B2",
        "B3 (TI 07)":"B3"    
    }

    df["tbclass"] = df["tbclass"].map(tbclass)
    
    return df

In [6]:

project = redcap.Project(redcap_api_url, pid_202_token)

In [7]:
response = project.import_records(df, force_auto_number=False)
response

{'count': 1}

# Appendix

In [8]:
#taking a glance at the data

import requests
import io
def redcap_API_export(url,token):
    """
    all data API request
    """

    fields = {
        'token': token,
        'content': 'record',
        'format': 'csv',
        'type': 'flat'}

    r = requests.post(url, data=fields)
    
    df = pd.read_csv(io.StringIO(r.content.decode("utf-8")), index_col=0)

    return df

df_pid202 = redcap_API_export(redcap_api_url, pid_202_token)